# Database Helper Examples

This notebook demonstrates the refreshed helpers in `ml.common.database`.
They mirror the usage documented in `docs/database/` and provide quick
recipes for pulling Guild Wars 2 trading data with absolute or relative
time windows.


In [2]:
import os
import sys
from datetime import UTC, datetime
from pathlib import Path

from gw2ml import DatabaseClient, database  # noqa: E402


## Environment setup

1. Ensure a `.env` file (or environment) provides `DB_URL` with access to the
   production replica described in `docs/database/public_tables.md`.
2. Optionally set `NOTEBOOK_ITEM_ID` if you want to analyse a different item ID.
3. Run the cells below inside the managed `uv` environment (`uv run --project ml jupyter notebook`).


In [3]:
client = DatabaseClient.from_env()
ITEM_ID = int(os.getenv("NOTEBOOK_ITEM_ID", "19702"))
ITEM_ID


19702

## Prices: relative lookback

Fetch the last week of prices for the current item. The helper accepts
`last_days`, `last_hours`, or `last_minutes` and automatically normalises
results to UTC.


In [4]:
prices_last_week = database.get_prices(
    client,
    item_id=ITEM_ID,
    last_days=60,
    # limit=300,
    order="DESC",
)
prices_last_week.head()


,id,item_id,whitelisted,buy_quantity,buy_unit_price,sell_quantity,sell_unit_price,fetched_at,created_at
0,791747235,19702,True,265352,180,696258,205,2025-11-21 12:30:47.973484+00:00,2025-11-21 12:30:47.973484+00:00
1,791720839,19702,True,265699,180,694388,208,2025-11-21 12:25:41.194322+00:00,2025-11-21 12:25:41.194322+00:00
2,791692443,19702,True,265555,180,694474,207,2025-11-21 12:20:37.122420+00:00,2025-11-21 12:20:37.122420+00:00
3,791664647,19702,True,265699,180,695180,205,2025-11-21 12:15:44.392015+00:00,2025-11-21 12:15:44.392015+00:00
4,791636251,19702,True,265809,180,694942,207,2025-11-21 12:10:28.376038+00:00,2025-11-21 12:10:28.376038+00:00


## Prices: explicit window

Supply timezone-aware start/end dates for a deterministic slice (mirrors
examples in `docs/database/prices_schema.md`).


In [5]:
custom_window_prices = database.get_prices(
    client,
    item_id=ITEM_ID,
    start_time=datetime(2025, 1, 1, tzinfo=UTC),
    end_time=datetime(2025, 1, 8, tzinfo=UTC),
    limit=500,
    order="DESC",
)
custom_window_prices.head()


,id,item_id,whitelisted,buy_quantity,buy_unit_price,sell_quantity,sell_unit_price,fetched_at,created_at


## BLTC history helpers

The BLTC table stores timestamps in seconds. The convenience wrapper handles
conversion and ordering automatically. Use `last_days` for quick pulls or
provide explicit `start_time` / `end_time` values.


In [6]:
bltc_last_three_days = database.get_bltc_history(
    client,
    item_id=ITEM_ID,
    last_days=3,
    limit=3000,
    order="DESC",
)
bltc_last_three_days.head()


,id,item_id,timestamp,sell_price,buy_price,supply,demand,sold,offers,bought,bids,created_at
0,49499683906,19702,2025-11-21 01:55:35+00:00,189,169,690445,258454,0,667,828,0,2025-11-21 02:10:41.743787+00:00
1,49499683905,19702,2025-11-21 01:42:19+00:00,189,173,689785,259282,0,2448,192,0,2025-11-21 02:10:41.743787+00:00
2,49499683904,19702,2025-11-21 01:27:53+00:00,192,169,695933,259474,0,0,0,0,2025-11-21 02:10:41.743787+00:00
3,49499683903,19702,2025-11-21 01:14:57+00:00,192,169,686651,259474,0,306,889,0,2025-11-21 02:10:41.743787+00:00
4,49499683902,19702,2025-11-21 00:59:25+00:00,187,170,686345,260363,0,691,275,0,2025-11-21 02:10:41.743787+00:00


In [7]:
bltc_custom_range = database.get_bltc_history(
    client,
    item_id=ITEM_ID,
    start_time=datetime(2025, 2, 1, tzinfo=UTC),
    end_time=datetime(2025, 2, 7, tzinfo=UTC),
    limit=5000,
    order="DESC",
)
bltc_custom_range.head()


,id,item_id,timestamp,sell_price,buy_price,supply,demand,sold,offers,bought,bids,created_at
0,165199915,19702,2025-02-07 00:00:00+00:00,181,158,533962,194860,13552,13715,7847,10062,2025-07-29 21:19:17.132938+00:00
1,165199914,19702,2025-02-06 18:00:00+00:00,186,158,535824,192752,10504,11749,7311,6185,2025-07-29 21:19:17.132938+00:00
2,165199913,19702,2025-02-06 12:00:00+00:00,187,169,536613,193129,8660,14978,6891,11243,2025-07-29 21:19:17.132938+00:00
3,165199912,19702,2025-02-06 06:00:00+00:00,180,155,528802,188777,9636,14471,9251,5322,2025-07-29 21:19:17.132938+00:00
4,165199911,19702,2025-02-06 00:00:00+00:00,191,164,524599,192706,15497,8918,11540,15129,2025-07-29 21:19:17.132938+00:00


## TP history helpers

`get_tp_history` mirrors the timed access patterns but honours the millisecond
precision used in `gw2tp_historical_prices`.


In [8]:
tp_last_hours = database.get_tp_history(
    client,
    item_id=ITEM_ID,
    last_hours=12,
    limit=4000,
    order="DESC",
)
tp_last_hours.head()


,id,item_id,timestamp,sell_price,buy_price,supply,demand,created_at


In [9]:
tp_custom_range = database.get_tp_history(
    client,
    item_id=ITEM_ID,
    start_time=datetime(2025, 3, 1, tzinfo=UTC),
    end_time=datetime(2025, 3, 5, tzinfo=UTC),
    limit=5000,
    order="DESC",
)
tp_custom_range.head()


,id,item_id,timestamp,sell_price,buy_price,supply,demand,created_at
0,124190883,19702,2025-03-04 08:33:53+00:00,208,174,542344,218288,2025-07-29 18:51:00.335576+00:00
1,124190882,19702,2025-03-04 02:49:15+00:00,202,169,546722,213411,2025-07-29 18:51:00.335576+00:00
2,124190881,19702,2025-03-03 08:18:27+00:00,204,157,545060,201775,2025-07-29 18:51:00.335576+00:00
3,124190880,19702,2025-03-02 19:43:10+00:00,200,162,547210,206938,2025-07-29 18:51:00.335576+00:00
4,124190879,19702,2025-03-02 08:58:34+00:00,198,161,555564,210778,2025-07-29 18:51:00.335576+00:00


## Optional exports

Everything returned is a `pandas.DataFrame`. Use your preferred persistence or
analysis pattern—e.g. writing to `data/tests/` for reproducible snapshots.


In [10]:
export_dir = Path("../data/tests").resolve()
export_dir.mkdir(parents=True, exist_ok=True)
(prices_last_week
 .to_csv(export_dir / "notebook_prices_last_week.csv", index=False))
